In [3]:
import ollama
import yfinance as yf

In [12]:
# Define a function to use as a tool
def get_stock_price(ticker):
    stock = yf.Ticker(ticker=ticker)
    return stock.history(period = '1d')["Close"].iloc[-1]

In [8]:
ollama.list()

ListResponse(models=[Model(model='phi3:latest', modified_at=datetime.datetime(2025, 2, 3, 14, 42, 44, 317737, tzinfo=TzInfo(+01:00)), digest='4f222292793889a9a40a020799cfd28d53f3e01af25d48e06c5e708610fc47e9', size=2176178913, details=ModelDetails(parent_model='', format='gguf', family='phi3', families=['phi3'], parameter_size='3.8B', quantization_level='Q4_0'))])

In [20]:
# Pass the function as a tool to Ollama
response = ollama.chat(
    model = "llama3.2",
    messages=[{"role":"user", "content":"What is the stock price of Apple?"}],
    tools={get_stock_price}
)
response.model_dump_json()

'{"model":"llama3.2","created_at":"2025-02-05T09:51:36.74879667Z","done":true,"done_reason":"stop","total_duration":279168947,"load_duration":17927137,"prompt_eval_count":154,"prompt_eval_duration":7000000,"eval_count":19,"eval_duration":252000000,"message":{"role":"assistant","content":"","images":null,"tool_calls":[{"function":{"name":"get_stock_price","arguments":{"ticker":"AAPL"}}}]}}'

In [14]:
# Call the function from the model response
available_functions = {
    'get_stock_price':get_stock_price
}

for tool in response.message.tool_calls or []:
    function_to_call = available_functions.get(tool.function.name)
    if function_to_call:
        print("Arguments", tool.function.arguments)
        print("Function output", function_to_call(**tool.function.arguments))
    else:
        print("Function not found",tool.function.name)

Arguments {'ticker': 'AAPL'}
Function output 232.8000030517578
